In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from zoofs import ParticleSwarmOptimization
import lightgbm as lgb
from sklearn.metrics import log_loss, accuracy_score, classification_report, f1_score

In [ ]:
df  = pd.read_csv()

In [ ]:
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
label_mapping = {
    'DDoS': 0,
    'DoS': 1,
    'Mirai': 2
}

df['label'] = df['label'].apply(lambda x: label_mapping.get(str(x), -1))  # Use -1 for unmatched labels if needed

In [ ]:
df.label.unique()

In [ ]:
X = df.drop(['label'],axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, shuffle=True,random_state = 0)

In [ ]:
def objective_function_topass(model,X_train, y_train, X_valid, y_valid):      
    model.fit(X_train,y_train)  
    P = log_loss(y_valid,model.predict_proba(X_valid))
    return P

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

def objective_function_topass(model, X_train, y_train, X_valid, y_valid):      
    """
    Objective function for optimization. Trains the model and evaluates F1 score.
    Returns the negative F1 score to maximize it.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)  
    P = f1_score(y_valid, y_pred, average='weighted')  # Weighted F1 score
    return -P  # Negative F1 score to maximize it

# Assuming ParticleSwarmOptimization is implemented and imported
algo_object = ParticleSwarmOptimization(
    objective_function_topass,
    n_iteration=15,
    population_size=30,
    minimize=True
)

# Replace LightGBM model with XGBoost
xgb_model = XGBClassifier(
    eval_metric="logloss"  # Evaluation metric for XGBoost
)

# Fit the optimization algorithm
algo_object.fit(xgb_model, X_train, y_train, X_test, y_test, verbose=True)

# Plot your results (if your ParticleSwarmOptimization class supports it)
# algo_object.plot()


In [ ]:
def objective_function_topass(model, X_train, y_train, X_valid, y_valid):      
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)  
    P = f1_score(y_valid, y_pred)
    return -P  # Return negative F1 score to maximize it

    
# create object of algorithm
algo_object_pso=ParticleSwarmOptimization(objective_function_topass,n_iteration=15,
                                    population_size=30,minimize=True)

lgb_model = lgb.LGBMClassifier()                                       
# fit the algorithm
algo_object_pso.fit(lgb_model,X_train, y_train, X_test, y_test,verbose=True)
#plot your results

In [ ]:
feat_xg = algo_object.best_feature_list
feat_lg = algo_object_pso.best_feature_list

In [ ]:
algo_object.plot_history()
algo_object_pso.plot_history()

In [ ]:
clf = lgb.LGBMClassifier()
clf.fit(X_train[feat_xg], y_train)
y_pred=clf.predict(X_test[feat_xg])

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
clf = lgb.LGBMClassifier()
clf.fit(X_train[feat_lg], y_train)
y_pred=clf.predict(X_test[feat_lg])

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
from zoofs import HarrisHawkOptimization

def objective_function_topass_harris(model,X_train, y_train, X_valid, y_valid):      
    model.fit(X_train,y_train)  
    P=f1_score(y_valid,model.predict_proba(X_valid))
    return -P

# import an algorithm !  


# create object of algorithm
algo_object_haris=HarrisHawkOptimization(objective_function_topass,
                                   n_iteration=10,
                                   population_size=30,
                                   minimize=True)

# fit the algorithm
algo_object_haris.fit(lgb_model,X_train, y_train, X_test, y_test,verbose=True)

#plot your results
algo_object_haris.plot_history()

# extract the best  feature set
algo_object_haris.best_feature_list 

In [ ]:
algo_object_haris.plot_history()

In [ ]:
algo_object_haris.best_feature_list 

In [ ]:
feat_hawk = algo_object_haris.best_feature_list

In [ ]:
clf = lgb.LGBMClassifier()
clf.fit(X_train[feat_hawk], y_train)
y_pred=clf.predict(X_test[feat_hawk])

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))